In [2]:
### PREREQUISTIES
### (many used only in one notebook...)

import os
import pandas as pd
import numpy as np
import logging ### to monitor the code
from bs4 import BeautifulSoup
import pickle
import math
import random
import sys
import csv
import unicodedata
import requests
import re

import nltk
from nltk.collocations import *

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

import xml.etree.cElementTree as ET
from urllib.request import urlopen

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
#from google.colab import auth
#from oauth2client.client import GoogleCredentials
from google.oauth2 import service_account # based on google-auth library
import sddk

In [2]:
#!pip install anda
#from anda import gr ### the import takes substantial time, since it import a +600MB file containing ancient Greek dictionary

In [3]:
### not neccessary for reading the data, just for exporting them to sciencedata.dk
conf = sddk.configure()

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
endpoint variable has been configured to: https://sciencedata.dk/files/


In [4]:
# to access gsheet, you need Google Service Account key json file
# I have mine located in my personal space on sciencedata.dk, so I read it from there:

# (1) read the file and parse its content
file_data = conf[0].get(conf[1] + "ServiceAccountsKey.json").json()
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)
# (5) establish connection with spreadsheets specified by their url
PIA_data = gc.open_by_url("https://docs.google.com/spreadsheets/d/1KxOx7Be9fj3lDcEPgQhQ-Iqcn9p367-MMD6RMXe8rks/edit?usp=sharing")
PIA_overview = gc.open_by_url("https://docs.google.com/spreadsheets/d/1e94wyelg6dftQ4zxbq1xvwxWAI-BhcYXtclDW-YTnrw/edit?usp=sharing")

In [5]:
publicfolder = "31b393e2afe1ee96ce81869c7efe18cb"
c_hippocraticum = sddk.read_file("c_hippocraticum_enriched.json", "df", publicfolder)

reading file located in a public folder


In [6]:
c_hippocraticum.columns

Index(['filename', 'author', 'title', 'string', 'wordcount', 'author_id',
       'doc_id', 'raw_date', 'date_avr', 'date_probs', 'date_manual',
       'provenience', 'lemmata', 'lemmata_wordcount', 'lemmatized_sentences',
       'n_sentences', 'tlg_epithet', 'lemmata_repl',
       'lemmatized_sentences_repl', 'λύπ*', 'ἄλγ*', 'ὀδύν*', 'πόνο*',
       'terms_sum', 'λύπ*_weighted', 'ἄλγ*_weighted', 'ὀδύν*_weighted',
       'πόνο*_weighted', 'weighted_sum', 'work_cat', 'work_cat_craik'],
      dtype='object')

In [7]:
words = ['λύπ*', 'ἄλγ*', 'ὀδύν*', 'πόνο*']

In [8]:
def sentences_with_term(sentences, term, flatlist=False):
    sentences_with_term = [sen for sen in sentences if term in sen]
    if flatlist == True:
        flatlist = [word for sen in sentences_with_term for word in sen]
        return flatlist
    else: 
        return sentences_with_term


In [9]:
for word in words:
    c_hippocraticum[word + "_sents"] = c_hippocraticum["lemmatized_sentences_repl"].apply(lambda x: sentences_with_term(x, word, True))

In [10]:
c_hippocraticum

,filename,author,title,string,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,...,ἄλγ*_weighted,ὀδύν*_weighted,πόνο*_weighted,weighted_sum,work_cat,work_cat_craik,λύπ*_sents,ἄλγ*_sents,ὀδύν*_sents,πόνο*_sents
1039,tlg0627.tlg001.perseus-grc1.xml,Hippocrates,De prisca medicina,ὁπόσοι μὲν ἐπεχείρησαν περὶ ἰητρικῆς λέγειν ἢ ...,5534,tlg0627,tlg0627.tlg001,5-4 B.C.,-4.0,"{'-4.5': 0.5, '-3.5': 0.5}",...,0.000181,0.000361,0.003072,0.004698,(Anti)philosophical,Scientific principles,"[κεκρημένα, ἀλλήλων, φανερός, λύπ*, ἄνθρωπος, ...","[χρῆ, οἶδα, τίσις, ῥυφήματα, νόσος, συμφέρω, π...","[ὀδύν*, καῦμα, φλογμός, ἔσχατος, καταχέω, ῥεῦμ...","[ζητέω, λέγω, προσήκω, πάθημα, νοσεουσί, πόνο*..."
1040,tlg0627.tlg002.perseus-grc1.xml,Hippocrates,De aëre aquis et locis,"ἰητρικὴν ὅστις βούλεται ὀρθῶς ζητεῖν, τάδε χρὴ...",7502,tlg0627,tlg0627.tlg002,5-4 B.C.,-4.0,"{'-4.5': 0.5, '-3.5': 0.5}",...,0.000000,0.000133,0.000133,0.000267,Itinerant physician,Scientific principles,[],[],"[ὁκόταν, ὀρεύς, στόμαχος, κύστις, προσπίτνω, ὅ...","[ἡσυχία, ῥᾳθυμίης, δειλίη, αὐξάνω, ταλαιπωρίης..."
1041,tlg0627.tlg003.perseus-grc1.xml,Hippocrates,Prognosticon,τὸν ἰητρὸν δοκέει μοι ἄριστον εἶναι πρόνοιαν ἐ...,5217,tlg0627,tlg0627.tlg003,5-4 B.C.,-4.0,"{'-4.5': 0.5, '-3.5': 0.5}",...,0.001533,0.004025,0.002875,0.008626,Prognostics,Cases and signs,"[ἥκιστος, λύπ*, κοιμάω, πρωϊ, τρίτος, μέρος, ἡ...","[ἐπανερωτᾷν, χρῆ, κεφαλή, ἄλγ*, ἀμβλυωπέουσιν,...","[γαστήρ, κεῖμαι, ξύνηθές, ὑγιαίνω, κοιμάω, παρ...","[πνεῦμα, πυκνός, πόνο*, σημαίνω, φλεγμονη, φρή..."
1042,tlg0627.tlg004.perseus-grc1.xml,Hippocrates,De diaeta in morbis acutis,"οἱ ξυγγράψαντες τὰς Κνιδίας καλεομένας γνώμας,...",6128,tlg0627,tlg0627.tlg004,5-4 B.C.,-4.0,"{'-4.5': 0.5, '-3.5': 0.5}",...,0.000653,0.002448,0.002121,0.005222,Prognostics,"Nosology, pathology, therapy",[],"[χράομαι, ποτός, ἢν, ἄλγ*, ἔχις, ὀξυμέλιτι, χε...","[ὀδύν*, πλευριτικοῖσιν, αὐτόματος, παύω, ἄρχω,...","[ἢν, λύω, θερμάσματα, πόνο*, χρῆ, πολὺν, χρόνο..."
1043,tlg0627.tlg005.perseus-grc1.xml,Hippocrates,De diaeta acutorum (Sp.),"καῦσος γίγνεται, ὁκόταν ἀναξηρανθέντα τὰ φλέβι...",5218,tlg0627,tlg0627.tlg005,5-4 B.C.,-4.0,"{'-4.5': 0.5, '-3.5': 0.5}",...,0.003450,0.000958,0.002300,0.007474,Other (Jouanna 2005:66),"Nosology, pathology, therapy","[ἄφωνος, γίγνομαι, φλέψ, ἀπόληψις, λύπ*, ἢν, ὑ...","[πυρετός, πολὺς, ἴσχω, σῶμα, κόπος, κοπιάω, ἄλ...","[ἢν, αἷμα, ῥίνη, ῥέω, ἀπόστημα, τράχηλος, γίγν...","[ὑποχόνδριον, ἐπίπονόν, ἔχις, σκολιότης, κόρος..."
1044,tlg0627.tlg006.perseus-grc1.xml,Hippocrates,Epidemiae,ἐν Θάσῳ φθινοπώρου περὶ ἰσημερίην καὶ ὑπὸ πληι...,10889,tlg0627,tlg0627.tlg006,5-4 B.C.,-4.0,"{'-4.5': 0.5, '-3.5': 0.5}",...,0.001102,0.002663,0.006153,0.010102,Epidemics,Cases and signs,"[θάσῳ, γυνή, δυσάνιος, λύπ*, πρόφασις, γίγνομα...","[κεφαλή, τράχηλος, ἄλγ*, βᾶρις, μετʼ, ὀδύν*, ἀ...","[πολύς, χρόνος, φλεγμοναί, μετʼ, ὀδύν*, ὄρχιν,...","[πολύς, περίρροιαι, πόνο*, χολώδης, ὑδατώδεες,..."
1045,tlg0627.tlg007.perseus-grc1.xml,Hippocrates,De capitis vulneribus,τῶν ἀνθρώπων αἱ κεφαλαὶ οὐδὲν ὁμοίως σφίσιν αὐ...,4988,tlg0627,tlg0627.tlg007,5-4 B.C.,-4.0,"{'-4.5': 0.5, '-3.5': 0.5}",...,0.000000,0.000000,0.000200,0.000200,Surgical,Surgery,[],[],[],"[διαμοτῶσαι, ἕλκος, πᾶς, μοτῷ, εὐρύς, ἕλκος, π..."
1046,tlg0627.tlg008.perseus-grc1.xml,Hippocrates,De officina medici,"ἢ ὅμοια ἢ ἀνόμοια ἐξ ἀρχῆς· ἀπὸ τῶν μεγίστων, ...",2145,tlg0627,tlg0627.tlg008,5-4 B.C.,-4.0,"{'-4.5': 0.5, '-3.5': 0.5}",...,0.000000,0.000000,0.000000,0.000000,not specified,Guidance and ideals,[],[],[],[]
1047,tlg0627.tlg009.perseus-grc1.xml,Hippocrates,De fracturis,ἐχρῆν τὸν ἰητρὸν τῶν ἐκπτωσίων τε καὶ κατηγμάτ...,11238,tlg0627,tlg0627.tlg009,5-4 B.C.,-4.0,"{'-4.5': 0.5, '-3.5': 0.5}",...,0.000089,0.000178,0.001068,0.001424,Surgical,Surgery,"[ἢν, οἶδα, ἱκανός, ὀστέον, ἀπιθύνω, προτέρῃσιν...","[ἄλγ*, ἐξέχω]","[γιγνώσκω, ἢν, ἕλκος, ὑπῇ, ὀδύν*, σφυγμοῖσιν, ...","[ἐπιδέω, ἔχω, χείρ, κελεύω, πόνο*, πολλοὺς, πρ..."
1048,tlg0627.tlg010.perseus-grc1.xml,Hippocrates,De articulis,

In [12]:
wordlists = {}
for word in words:
    corpus_wordlist = []
    for wordlist in c_hippocraticum[word + "_sents"]:
        corpus_wordlist.extend(wordlist)
    wordlists[word] = corpus_wordlist

In [27]:
wordlists_freqs = {}
for word in words:
    word_freqs = nltk.FreqDist(wordlists[word]).most_common(21)
    wordlists_freqs[word + ", ({0})".format(word_freqs[0][1]) ] = word_freqs[1:]
wordlists_freqs_df = pd.DataFrame(wordlists_freqs)
wordlists_freqs_df

,"λύπ*, (58)","ἄλγ*, (315)","ὀδύν*, (642)","πόνο*, (651)"
0,"(γίγνομαι, 14)","(ἢν, 78)","(ἢν, 198)","(ἢν, 124)"
1,"(ἢν, 14)","(πυρετός, 61)","(γίγνομαι, 153)","(γίγνομαι, 111)"
2,"(ἄνθρωπος, 12)","(κεφαλή, 61)","(ἔχις, 141)","(σῶμα, 102)"
3,"(σῶμα, 12)","(γίγνομαι, 51)","(πυρετός, 109)","(πολύς, 65)"
4,"(πολύς, 7)","(ὀδύν*, 46)","(κεφαλή, 89)","(κοιλία, 59)"
5,"(κοιλία, 7)","(κοιλία, 42)","(γαστήρ, 80)","(μαλλός, 58)"
6,"(ἡμέρα, 6)","(ἔχις, 39)","(πολύς, 66)","(πυρετός, 57)"
7,"(εἶμι, 6)","(γαστήρ, 35)","(ἴσχω, 62)","(πλείων, 56)"
8,"(ὀλίγος, 6)","(ὀσφύς, 28)","(ὁκόταν, 59)","(πᾶς, 55)"
9,"(χρόνος, 5)","(σῶμα, 26)","(κοιλία, 50)","(χρόνος, 45)"


In [23]:
set_with_dataframe(PIA_overview.add_worksheet("wordlists_freqs_df", 1, 1), wordlists_freqs_df)

In [28]:
most_common = []
for column in wordlists_freqs_df.columns:
    most_common.extend([tup[0] for tup in wordlists_freqs_df[column].tolist()])

In [31]:
list(set(most_common))

['ὀλίγος',
 'ὀσφύς',
 'δοκέω',
 'ψυχρός',
 'νόσος',
 'χολή',
 'ἄλγ*',
 'αἷμα',
 'χρόνος',
 'κενεών',
 'μαλλός',
 'ἡμέρα',
 'ἰσχυρός',
 'ὁκόταν',
 'συμβαίνω',
 'ἴσχω',
 'ἐσθίω',
 'σιτίον',
 'λαμβάνω',
 'πρότερος',
 'σῶμα',
 'δίδωμι',
 'ἔχις',
 'ἢν',
 'πλείων',
 'παύω',
 'γίγνομαι',
 'ποιέω',
 'ἄνθρωπος',
 'ἰξύς',
 'θερμός',
 'πόνο*',
 'γαστήρ',
 'πυρετός',
 'ὀδύν*',
 'κεφαλή',
 'σάρξ',
 'κοιλία',
 'εἶμι',
 'ὀξύς',
 'πᾶς',
 'πολύς',
 'μήτρα']